# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)

# load data

In [ ]:
from scipy.io import loadmat

data = loadmat('all_monkey_data/one_ff_data/sessions_python.mat',
               squeeze_me=True,
               struct_as_record=False)

sessions = data['sessions_out']


## behavioral data

In [ ]:
trial_num = 30
trial_data = sessions[0].behaviour_trials[trial_num].continuous
print(trial_data._fieldnames)

x = trial_data.xmp
y = trial_data.ymp
v = trial_data.v
w = trial_data.w
t = trial_data.ts


In [ ]:
trial_events = sessions[0].behaviour_trials[trial_num].events

In [ ]:
trial_events.t_flyON

In [ ]:
dt = np.average(np.diff(t))
dt


In [ ]:
import matplotlib.pyplot as plt

plt.plot(x, y, 'k-')
plt.xlabel('x (forward)')
plt.ylabel('y (lateral)')
plt.axis('equal')


## neural data

In [ ]:
# get unit0's trial_0 spike times
unit0 = sessions[0].units[0]
trial0 = unit0.trials[0]
trial0._fieldnames
trial0.tspk


# concat data

In [ ]:
session = sessions[0]

In [ ]:
trials = session.behaviour_trials
units = session.units

In [ ]:

prs.dt
full_time_window(tr)
bin_spikes(...)
concatenate_trials(...)
trial_ids = np.arange(n_trials)


In [ ]:
def concatenate_trials_with_trial_id(trials, trial_ids, signal_fn, time_window_fn):
    X = []
    trial_id_vec = []

    for tid in trial_ids:
        tr = trials[tid]
        mask = time_window_fn(tr)

        sig = signal_fn(tr, tid)[mask]
        X.append(sig)

        trial_id_vec.append(np.full(len(sig), tid, dtype=int))

    return np.concatenate(X), np.concatenate(trial_id_vec)


In [ ]:
prs.posttrial

In [ ]:
v_concat, trial_id_vec = concatenate_trials_with_trial_id(
    trials,
    trial_ids,
    lambda tr, tid: tr.continuous.v,
    full_time_window
)


In [ ]:
k = 0  # neuron to fit

In [ ]:
spk_counts, trial_id_vec_spk = concatenate_trials_with_trial_id(
    trials,
    trial_ids,
    lambda tr, tid: bin_spikes(
        units[k].trials[tid].tspk,
        tr.continuous.ts
    ),
    full_time_window
)


In [ ]:
trial_id_vec_spk.shape

In [ ]:
def event_impulse(tr, tid, event_name):
    ts = tr.continuous.ts
    evt_t = getattr(tr.events, event_name)

    ev = np.zeros_like(ts)
    idx = np.searchsorted(ts, evt_t)
    if 0 <= idx < len(ts):
        ev[idx] = 1.0
    return ev

events_concat, trial_id_vec_evt = concatenate_trials_with_trial_id(
    trials,
    trial_ids,
    lambda tr, tid: event_impulse(tr, tid, 't_targ'),
    full_time_window
)


In [ ]:
for events in ['t_targ', 't_move', 't_rew']:
    events_concat, trial_id_vec_evt = concatenate_trials_with_trial_id(
        trials,
        trial_ids,
        lambda tr, tid: event_impulse(tr, tid, 't_targ'),
        full_time_window
    )

# Replicate (chatGPT)

In [ ]:

# =========================
# Load data
# =========================

data = loadmat(
    'all_monkey_data/one_ff_data/sessions_python.mat',
    squeeze_me=True,
    struct_as_record=False
)

sessions = data['sessions_out']
session = sessions[0]

trials = session.behaviour_trials
units = session.units

n_trials = len(trials)
n_units = len(units)


# =========================
# Build trial index (all trials)
# =========================

trial_ids = np.arange(n_trials)



In [ ]:
"""
Population analysis pipeline
Python replication of AnalysePopulation.m (core functionality)

Author: you + ChatGPT
"""

import numpy as np
from scipy.io import loadmat
from scipy.signal import hilbert
from scipy.signal.windows import gaussian
from scipy.linalg import lstsq
from sklearn.cross_decomposition import CCA
from dataclasses import dataclass
from typing import List, Dict


# =========================
# Parameters (prs struct)
# =========================

@dataclass
class Params:
    dt: float = 0.01
    neural_filtwidth: int = 5
    pretrial: float = 0
    posttrial: float = 0
    cca_vars: List[str] = None
    decode_vars: List[str] = None


prs = Params(
    dt=0.01,
    neural_filtwidth=5,
    cca_vars=['v', 'w'],
    decode_vars=['v', 'w']
)


# =========================
# Utilities
# =========================

def gaussian_kernel(width):
    t = np.arange(-2 * width, 2 * width + 1)
    h = np.exp(-t ** 2 / (2 * width ** 2))
    return h / h.sum()


def smooth_signal(x, width):
    if width <= 0:
        return x
    h = gaussian_kernel(width)
    return np.apply_along_axis(lambda m: np.convolve(m, h, mode='same'), 0, x)


def bin_spikes(spike_times, ts):
    counts = np.zeros(len(ts))
    idx = np.searchsorted(ts, spike_times)
    idx = idx[(idx >= 0) & (idx < len(ts))]
    np.add.at(counts, idx, 1)
    return counts


def concatenate_trials(trials, trial_ids, signal_fn, time_window_fn):
    X = []
    trial_lengths = []

    for tid in trial_ids:
        tr = trials[tid]
        mask = time_window_fn(tr)
        sig = signal_fn(tr, tid)[mask]   # <-- pass tid
        X.append(sig)
        trial_lengths.append(len(sig))

    return np.concatenate(X), trial_lengths


def deconcatenate(x, trial_lengths):
    out = []
    idx = 0
    for L in trial_lengths:
        out.append(x[idx:idx+L])
        idx += L
    return out


def compute_d(v, ts, dt):
    d = np.zeros_like(v)
    valid = ts > 0
    d[valid] = np.cumsum(v[valid]) * dt
    return d


def compute_phi(w, ts, dt):
    phi = np.zeros_like(w)
    valid = ts > 0
    phi[valid] = np.cumsum(w[valid]) * dt
    return phi


# =========================
# Time window helper
# =========================

def full_time_window(tr):
    t0 = min(tr.events.t_move, tr.events.t_targ) - prs.pretrial
    t1 = tr.events.t_end + prs.posttrial
    return (tr.continuous.ts >= t0) & (tr.continuous.ts <= t1)


# =========================
# Build stimulus matrix X
# =========================

def get_var(tr, name):
    if name == 'v':
        return tr.continuous.v
    if name == 'w':
        return tr.continuous.w
    if name == 'd':
        return compute_d(tr.continuous.v, tr.continuous.ts, prs.dt)
    if name == 'phi':
        return compute_phi(tr.continuous.w, tr.continuous.ts, prs.dt)
    raise ValueError(name)


def gen_traj(w, v, ts):
    x = np.zeros(len(ts))
    y = np.zeros(len(ts))
    for i in range(1, len(ts)):
        x[i] = x[i-1] + v[i] * np.cos(w[i]) * prs.dt
        y[i] = y[i-1] + v[i] * np.sin(w[i]) * prs.dt
    return x, y




# run

In [ ]:
X_list = []
trial_lengths = None
total_len_ref = None

for var in prs.cca_vars:
    x, tl = concatenate_trials(
        trials,
        trial_ids,
        lambda tr, tid, v=var: get_var(tr, v),
        full_time_window
    )

    if trial_lengths is None:
        trial_lengths = tl
        total_len_ref = x.shape[0]
    else:
        if x.shape[0] != total_len_ref:
            raise RuntimeError(f'Concatenated length mismatch for var {var}: {x.shape[0]} vs {total_len_ref}')

    X_list.append(x)

X = np.column_stack(X_list)
X[np.isnan(X)] = 0


In [ ]:


# # =========================
# # Build population activity Y
# # =========================

# Y = np.zeros((X.shape[0], n_units))

# for k in range(n_units):
#     yk, _ = concatenate_trials(
#         trials,
#         trial_ids,
#         lambda tr, tid, k=k: bin_spikes(
#             units[k].trials[tid].tspk,
#             tr.continuous.ts
#         ),
#         full_time_window
#     )
#     Y[:, k] = yk

# Y_smooth = smooth_signal(Y, prs.neural_filtwidth) / prs.dt


# # =========================
# # Canonical Correlation Analysis
# # =========================

# cca = CCA(n_components=min(X.shape[1], Y_smooth.shape[1]))
# Xc, Yc = cca.fit_transform(X, Y_smooth)

# cca_corrs = [np.corrcoef(Xc[:, i], Yc[:, i])[0, 1] for i in range(Xc.shape[1])]

# print('CCA correlations:', cca_corrs)


# # =========================
# # Linear population decoding
# # =========================

# decode_results = {}

# for var in prs.decode_vars:
#     xt, _ = concatenate_trials(
#         trials,
#         trial_ids,
#         lambda tr, tid, v=var: get_var(tr, v),
#         full_time_window
#     )
#     xt[np.isnan(xt)] = 0

#     Yd = smooth_signal(Y, prs.neural_filtwidth)

#     wts, _, _, _ = lstsq(Yd, xt)
#     pred = Yd @ wts

#     corr = np.corrcoef(xt, pred)[0, 1]

#     decode_results[var] = dict(
#         weights=wts,
#         corr=corr,
#         true=deconcatenate(xt, trial_lengths),
#         pred=deconcatenate(pred, trial_lengths)
#     )

#     print(f'Decode {var}: r = {corr:.3f}')


# # =========================
# # Trajectory reconstruction
# # =========================

# traj_pred = []

# for tr_v, tr_w in zip(decode_results['v']['pred'], decode_results['w']['pred']):
#     x, y = gen_traj(tr_w, tr_v, np.arange(len(tr_v)) * prs.dt)
#     traj_pred.append((x, y))


# print('Pipeline complete.')


# PGAM

In [ ]:
# Import libraries

# import sys
# ## if working outside the docker container, uncomment the line below and add the path to [YOUR PATH TO PGAM FOLDER]/src/
# ## sys.path.append('[YOUR PATH TO PGAM FOLDER]/src/')
# sys.path.append('src/')

pgam_path = '/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/external/pgam/src/'
import sys
if not pgam_path in sys.path: 
    sys.path.append(pgam_path)
    
import numpy as np
import sys
from PGAM.GAM_library import *
import PGAM.gam_data_handlers as gdh
import matplotlib.pylab as plt
import pandas as pd
from post_processing import postprocess_results
from scipy.io import savemat

## temporal covariance

Temporal filters g were parameterized using a basis of ten raised cosine filters spanning a range of 600 milliseconds. The filter associated with target-onset was causal ([0, 600] ms), while the remaining filters were non-causal ([-300, 300] ms). Both spike-history filter h and coupling filter p were expressed using a basis of ten causal raised cosine filters in logarithmic time scale. Spike-history filters spanned 350 ms, while coupling filters spanned 1.375 seconds.

In [ ]:
order = 4
knots_num = 10 - order
dt_ms = 1.0          # e.g. 1 ms bins (1 kHz sampling)
kernel_ms = 600
kernel_h_length = int(kernel_ms / dt_ms)


In [ ]:
tot_tp = 10**3

# # trial ids
# trial_ids = np.zeros(tot_tp)
# trial_ids[400:] = 1

# # event markers
# event = np.zeros(tot_tp)
# event[[100, 200, 600, 900]] = 1

# kernel parameters
dt_ms = 1.0                 # ms per time bin
kernel_ms = 600             # total temporal span
kernel_h_length = int(kernel_ms / dt_ms)

order = 4                   # cubic B-splines
num_filters = 10
num_int_knots = num_filters - order

dict_kernel = {
    0: 'Acausal',
    1: 'Direction %d' % 1,
    -1: 'Direction %d' % (-1)
}

for kernel_direction in [0, 1, -1]:
    sm_handler = gdh.smooths_handler()

    sm_handler.add_smooth(
        'this_event',
        [event],
        is_temporal_kernel=True,
        ord=order,
        knots_num=num_int_knots,
        trial_idx=trial_ids,
        kernel_length=kernel_h_length,
        kernel_direction=kernel_direction
    )
